In [1]:
import pandas as pd
import numpy as np

from keras import Sequential
from keras.layers import LSTM,Dense,Dropout

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.decomposition import FastICA


Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
file = pd.read_csv('Raw_TrainTest_MultiTasks.csv')
df = pd.DataFrame(file)
df.head(3)

,AF3,F7,F3,FC5,T7,P7,O1,O2,P8,T8,FC6,F4,F8,AF4,class_
0,4584.62,3902.05,4571.79,4589.23,4124.62,3825.13,4152.82,4579.49,4690.77,4260.00,4027.18,4385.13,4480.51,4230.77,2
1,4584.10,3895.90,4574.87,4567.69,4124.10,3827.18,4157.95,4585.13,4695.38,4268.21,4034.36,4380.00,4501.54,4197.44,2
2,4574.36,3893.85,4576.92,4572.82,4123.59,3829.23,4165.13,4590.26,4702.56,4281.54,4030.77,4366.67,4521.03,4176.41,2


In [3]:
ica = FastICA()
df_ica = pd.DataFrame(ica.fit_transform(df.drop('class_',axis=1)),columns=df.columns[:-1])
df_ica.head(3)

,AF3,F7,F3,FC5,T7,P7,O1,O2,P8,T8,FC6,F4,F8,AF4
0,-0.000816,-0.000246,-0.001308,0.000346,0.001592,-0.000020,-0.000105,0.000781,-0.001112,-0.000289,0.000284,0.000452,0.001136,-0.000370
1,-0.000630,-0.000221,-0.001283,0.000301,0.001523,-0.000115,-0.000148,0.000797,-0.001127,-0.000362,0.000452,0.000401,0.001186,-0.000565
2,-0.000562,-0.000189,-0.001283,0.000304,0.001602,-0.000100,-0.000174,0.000958,-0.001153,-0.000336,0.000629,0.000422,0.001231,-0.000725


In [4]:
x_train,x_test,y_train,y_test = train_test_split(df_ica,df['class_'],test_size=0.2)

x_train = np.reshape(np.array(x_train),(x_train.shape[0],1,x_train.shape[1]))
x_test = np.reshape(np.array(x_test),(x_test.shape[0],1,x_test.shape[1]))

y_train=np.reshape(np.array(y_train),(y_train.shape[0],1))
y_test=np.reshape(np.array(y_test),(y_test.shape[0],1))

In [5]:
model = Sequential()

In [6]:
model.add(LSTM(units=50,return_sequences=True,input_shape=(1,14),activation='sigmoid'))
model.add(Dropout(0.2))

model.add(LSTM(units=50,return_sequences=True,activation='sigmoid'))
model.add(Dropout(0.2))

model.add(LSTM(units=50,activation='sigmoid'))
model.add(Dropout(0.2))

model.add(Dense(units=1,activation='softmax'))

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [11]:
model.fit(x_train,y_train,verbose=1,epochs=10,batch_size=150)

Epoch 1/10
737280/737280 [==============================] - 97s 131us/step - loss: -8.3032 - accuracy: 0.4585
Epoch 2/10
737280/737280 [==============================] - 96s 130us/step - loss: -8.3032 - accuracy: 0.4585
Epoch 3/10
737280/737280 [==============================] - 95s 129us/step - loss: -8.3032 - accuracy: 0.4585
Epoch 4/10
737280/737280 [==============================] - 96s 130us/step - loss: -8.3032 - accuracy: 0.4585
Epoch 5/10
737280/737280 [==============================] - 95s 129us/step - loss: -8.3032 - accuracy: 0.4585
Epoch 6/10
737280/737280 [==============================] - 95s 129us/step - loss: -8.3032 - accuracy: 0.4585
Epoch 7/10
737280/737280 [==============================] - 94s 128us/step - loss: -8.3032 - accuracy: 0.4585
Epoch 8/10
737280/737280 [==============================] - 94s 127us/step - loss: -8.3032 - accuracy: 0.4585
Epoch 9/10
737280/737280 [==============================] - 94s 128us/step - loss: -8.3032 - accuracy: 0.4585
Epoch 10/1

In [12]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 1, 50)             13000     
_________________________________________________________________
dropout_1 (Dropout)          (None, 1, 50)             0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 1, 50)             20200     
_________________________________________________________________
dropout_2 (Dropout)          (None, 1, 50)             0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 50)                20200     
_________________________________________________________________
dropout_3 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                

In [13]:
model.evaluate(x_test,y_test)

184320/184320 [==============================] - 17s 95us/step


[-8.26910484958854, 0.4577365517616272]

In [14]:
model.predict_classes(x_test,verbose=1)

184320/184320 [==============================] - 12s 66us/step


array([[1],
       [1],
       [1],
       ...,
       [1],
       [1],
       [1]], dtype=int32)